<a href="https://colab.research.google.com/github/ItWasAllYellow/NLP_2025/blob/main/notebooks/1_word2vec.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Word2Vec Implementation from Scratch

This notebook demonstrates how to implement the Word2Vec algorithm from scratch using PyTorch. We'll use the first Harry Potter book as our corpus to train word embeddings.


## 1. Setting Up the Environment

First, we need to import the necessary libraries:
- `torch` and `torch.nn` for tensor operations and neural network functionality
- `string` for string manipulations (removing punctuation)


In [1]:
import torch
import torch.nn as nn
import string


## 2. Getting the Text Data

We'll download the first Harry Potter book to use as our corpus.

In [2]:
!wget "https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt"


--2025-03-18 05:25:14--  https://raw.githubusercontent.com/amephraim/nlp/master/texts/J.%20K.%20Rowling%20-%20Harry%20Potter%201%20-%20Sorcerer's%20Stone.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 439742 (429K) [text/plain]
Saving to: ‘J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt’

J. K. Rowling - Har 100%[===================>] 429.44K  --.-KB/s    in 0.05s   

2025-03-18 05:25:14 (8.90 MB/s) - ‘J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt’ saved [439742/439742]



## 3. Text Preprocessing

Before we can use the text data, we need to preprocess it:
- Remove punctuation
- Convert text to lowercase
- Split text into tokens (words)

This function will help us clean and tokenize the text.

In [3]:
def remove_punctuation(x):
  return x.translate(''.maketrans('', '', string.punctuation))

def make_tokenized_corpus(corpus):
  out= [ [y.lower() for y in remove_punctuation(sentence).split(' ') if y] for sentence in corpus]
  return [x for x in out if x!=[]]


## 4. Loading and Formatting the Text

Now we'll load the text file, replace some special characters, and split the text into sentences.


In [4]:
with open("J. K. Rowling - Harry Potter 1 - Sorcerer's Stone.txt", 'r') as f:
  strings = f.readlines()
sample_text = "".join(strings).replace('\n', ' ').replace('Mr.', 'mr').replace('Mrs.', 'mrs').split('. ')


Let's tokenize the text using our preprocessing function `make_tokenized_corpus`:

In [5]:
# Corpus is a list of list of strings (words)

In [6]:
corpus = make_tokenized_corpus(sample_text)

corpus[:5]

[['harry',
  'potter',
  'and',
  'the',
  'sorcerers',
  'stone',
  'chapter',
  'one',
  'the',
  'boy',
  'who',
  'lived',
  'mr',
  'and',
  'mrs',
  'dursley',
  'of',
  'number',
  'four',
  'privet',
  'drive',
  'were',
  'proud',
  'to',
  'say',
  'that',
  'they',
  'were',
  'perfectly',
  'normal',
  'thank',
  'you',
  'very',
  'much'],
 ['they',
  'were',
  'the',
  'last',
  'people',
  'youd',
  'expect',
  'to',
  'be',
  'involved',
  'in',
  'anything',
  'strange',
  'or',
  'mysterious',
  'because',
  'they',
  'just',
  'didnt',
  'hold',
  'with',
  'such',
  'nonsense'],
 ['mr',
  'dursley',
  'was',
  'the',
  'director',
  'of',
  'a',
  'firm',
  'called',
  'grunnings',
  'which',
  'made',
  'drills'],
 ['he',
  'was',
  'a',
  'big',
  'beefy',
  'man',
  'with',
  'hardly',
  'any',
  'neck',
  'although',
  'he',
  'did',
  'have',
  'a',
  'very',
  'large',
  'mustache'],
 ['mrs',
  'dursley',
  'was',
  'thin',
  'and',
  'blonde',
  'and',
  'had

## 5. Creating Context Word Pairs

A key concept in Word2Vec is learning from context. We need to create pairs of words that appear near each other in the text. We'll use a sliding window approach to create these pairs.

For example, with the window size of 2, for the word "to" in the sentence "they were the last people youd expect to be involved...", we would create pairs with:
- ("to", "expect")
- ("to", "be")
- ("to", "involved")
- ("to", "in")

These pairs will be our training data.

In [7]:
from tqdm import tqdm

sample_sentence = ['they', 'were', 'the', 'last', 'people', 'youd', 'expect', 'to', 'be', 'involved', 'in', 'anything', 'strange', 'or', 'mysterious', 'because', 'they', 'just', 'didnt', 'hold', 'with', 'such', 'nonsense']

word_pairs = []
window_size = 2

for sample_sentence in tqdm(corpus):
  for cur_idx, center_word in enumerate(sample_sentence):
    window_begin = max(cur_idx - window_size, 0)
    window_end = min(cur_idx + window_size + 1, len(sample_sentence))
    # for context_word in sample_sentence[window_begin:window_end]:
    #   # if center_word == context_word: continue
    #   word_pairs.append( (center_word, context_word))
    for j in range(window_begin, window_end):
      if cur_idx == j: continue
      word_pairs.append( (center_word, sample_sentence[j]))

print(f"\nLength of word_pairs is {len(word_pairs)}")
print(f"First 5 example of word_pairs is {word_pairs[:5]}")

100%|██████████| 4682/4682 [00:00<00:00, 5923.95it/s]


Length of word_pairs is 282372
First 5 example of word_pairs is [('harry', 'potter'), ('harry', 'and'), ('potter', 'harry'), ('potter', 'and'), ('potter', 'the')]


## 6. Building the Vocabulary

To work with word vectors, we need to create a vocabulary that maps each unique word to an index. We'll also filter out rare words that appear less than a certain number of times in the corpus.

### 6.1 Collecting All Words

First, let's collect all words in our corpus:


In [8]:
# we have to make vocabulary
sentence = corpus[0]
entire_words = []

for sentence in corpus:
  for word in sentence:
    entire_words.append(word)




### 6.2 Finding Unique Words

Now, let's find the unique words in our corpus:

In [9]:
# we have to get the "unique" item among total words

unique_words = set(entire_words)
len(unique_words)


6038

### 6.3 Converting to a List and Sorting

We'll convert the set of unique words to a sorted list:

In [10]:
# vocab_set[0] # set is not subscriptable because it has no order

unique_words = sorted(list(unique_words))
unique_words[0]


'\the'

### 6.4 Filtering by Frequency

Now, let's filter out rare words that occur less than a specified number of times:
- We can use the `Counter` class from the `collections` module to count the frequency of each word in the corpus.
- Caution on `alist.sort()` will return `None`.

In [11]:
# how can we filter the vocab by its frequency?
filtered_vocab = None
# you can use word counter as dictionary
# In python dictionary, dict.keys() gives keys, and dict.values() give values,
# dict.items() give (key, value)

from collections import Counter
word_counter = Counter(entire_words)
word_counter.most_common(10)
word_counter['harry']

threshold = 5
filtered_vocab = []

for key, value in word_counter.items():
  if value > threshold:
    filtered_vocab.append(key)

filtered_vocab.sort()
filtered_vocab[:10]

['a',
 'able',
 'abou',
 'about',
 'above',
 'across',
 'added',
 'afford',
 'afraid',
 'after']

## 7. Filtering Word Pairs

Now that we have our filtered vocabulary, we need to filter our word pairs to only include words that are in our vocabulary:

In [12]:
# Filter the word_pairs using the vocab
# word_pairs, filtered_vocab
# word_pairs is a list of [word_a, word_b]

filtered_word_pairs = []
vocab_set = set(filtered_vocab)

for pair in tqdm(word_pairs):
  a, b = pair
  if a in vocab_set and b in vocab_set:
    filtered_word_pairs.append(pair)


100%|██████████| 282372/282372 [00:00<00:00, 358023.85it/s]


In [13]:
# implement same algorithm with list comprehension

filtered_word_pairs = [pair for pair in word_pairs if pair[0] in vocab_set and pair[1] in vocab_set]


In [14]:
len(filtered_word_pairs), len(word_pairs)

(226846, 282372)

## 8. Converting Words to Indices

For efficiency, we'll convert our words to indices according to their position in our vocabulary:

In [15]:
# convert word into index of vocab
filtered_vocab.index('harry')

527

This is inefficient because `list.index()` has to scan the list every time. Let's use a dictionary for faster lookups:

In [16]:
# we can make it faster
# use dictionary to find the index of string
word2idx = dict()
for idx, word in enumerate(filtered_vocab):
  word2idx[word] = idx
word2idx['harry']

527

Now, let's convert our word pairs to index pairs more efficiently:

In [19]:
index_pairs = [(word2idx[pair[0]], word2idx[pair[1]]) for pair in filtered_word_pairs]
index_pairs[0]

(527, 953)

In [20]:
# Why we don't need idx2tok?

filtered_vocab[527]

'harry'

## 9. Creating Initial Word Vectors

Now we'll create random vectors for each word in our vocabulary. These vectors will be adjusted during training:
- We can use `torch.randn` to create random vectors that follow normal distribution.

In [26]:
# we have to make random vectors for each word in the vocab
# we also have to decide the dimension of the vector

dim = 100
vocab_size = len(filtered_vocab)

word_vectors = torch.randn(vocab_size, dim) / 10 # 일반적으로 벡터의 의미를 담고 있는 차원들을 뒤로 뺀다, 특정 단어의 벡터를 빼기 쉽기 때문
word_vectors

tensor([[-0.1442, -0.0750,  0.1677,  ...,  0.0481, -0.0059, -0.0494],
        [-0.0340, -0.0409, -0.0938,  ..., -0.1039,  0.0883, -0.0281],
        [-0.1188, -0.0660, -0.1107,  ..., -0.0919, -0.1359, -0.0358],
        ...,
        [-0.0728,  0.1432,  0.0942,  ...,  0.0631,  0.0526, -0.0775],
        [ 0.1029, -0.0082,  0.0629,  ..., -0.1282, -0.0532, -0.1945],
        [ 0.0406, -0.0301, -0.0545,  ..., -0.0331, -0.1650,  0.1634]])

In [29]:
# what is the vector for harry?
word_vectors[word2idx['harry']] # 몇 숫자만 크게 된다면 문제가 있음 -> 정규화?

tensor([     0.1383,     -0.1849,      0.0289,     -0.1807,     -0.1694,
             0.1503,     -0.1115,     -0.1284,     -0.0664,      0.0216,
             0.0307,     -0.0074,     -0.0696,      0.1945,      0.0672,
             0.1616,     -0.1016,      0.1191,      0.0605,      0.0309,
             0.1388,      0.0398,     -0.0562,     -0.0255,     -0.0543,
            -0.0199,      0.0071,      0.0043,     -0.0615,     -0.0057,
            -0.1522,     -0.0820,     -0.0621,      0.0530,      0.0062,
             0.0080,      0.0404,     -0.0975,     -0.1607,     -0.0184,
             0.1792,      0.0993,     -0.0479,      0.0652,      0.0346,
             0.0949,      0.0040,     -0.0192,      0.1256,      0.1697,
            -0.1190,     -0.2193,      0.1733,     -0.0194,     -0.1189,
            -0.1004,     -0.0499,      0.0284,     -0.0251,     -0.1820,
             0.2407,     -0.0303,     -0.0702,     -0.0674,     -0.0196,
             0.1770,      0.1054,     -0.0328,     

## 10. Understanding Word Relationships with Dot Products

The core of Word2Vec is using dot products to measure relationships between words. Let's explore this concept:

In [28]:
torch.set_printoptions(sci_mode=False) # Do this to avoid scientific notation


## Dot Product
- Assume we have two vectors $a$ and $b$.
  - $a = [a_1, a_2, a_3, a_4, ..., a_n]$
  - $b = [b_1, b_2, b_3, b_4, ..., b_n]$
- $a \cdot b$ = $\sum _{i=1}^n a_ib_i$  = $a_1b_1 + a_2b_2 + a_3b_3 + a_4b_4 + ... + a_nb_n$

Let's calculate the dot product between "harry" and "potter":


In [33]:
# calculate P(potter|harry)
harry = word_vectors[word2idx['harry']]
potter = word_vectors[word2idx['potter']]

dot_product_value_between_potter_harry = torch.dot(harry, potter) # sum(harry * potter)
dot_product_value_between_potter_harry

tensor(0.1538)

In [34]:
# we can get the dot product value for every other words in the vocab
# to get  P(word | harry)
word_dot_dict = {}
for word in filtered_vocab:
    w_idx = word2idx[word]
    w_vector = word_vectors[w_idx]
    word_dot_dict[word] = torch.dot(w_vector, harry)

word_dot_dict

{'a': tensor(0.2376),
 'able': tensor(-0.2906),
 'abou': tensor(0.0773),
 'about': tensor(0.0401),
 'above': tensor(0.0242),
 'across': tensor(-0.1567),
 'added': tensor(-0.0084),
 'afford': tensor(0.0220),
 'afraid': tensor(0.0609),
 'after': tensor(-0.0576),
 'afternoon': tensor(-0.0768),
 'again': tensor(-0.2382),
 'against': tensor(0.0317),
 'ages': tensor(0.0827),
 'ago': tensor(0.0136),
 'agreed': tensor(0.0270),
 'ah': tensor(-0.0181),
 'ahead': tensor(0.0321),
 'air': tensor(-0.1077),
 'albus': tensor(-0.0793),
 'alive': tensor(-0.0183),
 'all': tensor(-0.0816),
 'alley': tensor(-0.2130),
 'allowed': tensor(-0.1142),
 'almost': tensor(-0.0177),
 'alone': tensor(0.0993),
 'along': tensor(-0.1126),
 'already': tensor(-0.0928),
 'also': tensor(-0.0149),
 'although': tensor(0.0886),
 'always': tensor(-0.0046),
 'am': tensor(0.0780),
 'an': tensor(-0.0451),
 'and': tensor(-0.1499),
 'angrily': tensor(0.0850),
 'angry': tensor(0.1341),
 'another': tensor(-0.0926),
 'answer': tensor(-

Now, let's convert these dot products to probabilities using the softmax function:
- We have to convert our prediction into probability distribution to get P(word|harry) so that sum of [P(a|harry), ..., P(potter|harry), ... P(ron|harry), ... ] = 1
- current dot product value is any real number, sometimes called as logit
  - logit from logistic regression. Some values that are not yet converted to 0-1 or value before sigmoid function
  - every probability should be in range (0, 1) (greater than 0, smaller than 1)
  - this can be handled by taking exponential of dot product values, divided by total sum
  - This function is called **Softmax**

- Why we use exponential?
  - Because we want to make every probability in positive range while preserving the order


In [ ]:
from math import exp

word_exp_dict = {}
for word, dot_value in word_dot_dict.items():
    exp_value = None

word_prob_dict = None

In [ ]:
# Get P(potter|harry)

## 13. Efficient Matrix Operations
![img](https://mkang32.github.io/images/python/khan_academy_matrix_product.png)

Instead of calculating dot products one by one, we can use matrix multiplication for efficiency:


In [ ]:
# get dot product result for every word in the vocabulary

# first, make vector_of_harry into matrix format

# do matrix multiplication


Let's verify that our matrix multiplication gives the same result as individual dot products:

Now let's implement the complete softmax calculation using matrix operations:


In [ ]:
# convert dot product result into exponential

In [ ]:
# get the sum of exponential


In [ ]:
# divide exponential value with sum

## 14. Creating a Probability Function

Let's create a function to calculate probabilities efficiently:

In [ ]:
def get_probs(query_vectors, entire_vectors):
  return None

# get_probs(mat_of_harry, word_vectors)

## 15. Preparing for Training

Before training our Word2Vec model, we need to split our dataset into training and testing sets:

In [ ]:
# Now we can train the word2vec

# Let's think about training pairs
index_pairs # this is our dataset. It's list of list of two integer
# two integer means a pair of neighboring words

# Training set and Test set
# To validate that our model can solve 'unseen' problems
# So we have to split the dataset before training.

# To randomly split the dataset, we will first shuffle the dataset

# random.shuffle(index_pairs) # this will shuffle the list items

In [ ]:
len(train_set), len(test_set)

## 16. Training the Word2Vec Model

Now we'll train our Word2Vec model using batched gradient descent:

In [ ]:
# making batch from train_set
# Batch is a set of training samples, that are calculated together
# And also we update the model after one single batch

## 17. Evaluating the Training

Let's visualize the training loss to see if our model is learning:

In [ ]:
import matplotlib.pyplot as plt
plt.plot(loss_record)

## 18. Testing the Model

Now we'll test our model on the test set:

## 19. Exploring Learned Word Relationships

Let's explore what our model has learned by finding the words most closely related to "harry":

In [ ]:
# P(potter|harry)?
